# STEPS FOR KNN 
1. ## import data and indicate x-y values
2. ## split the data into train and test datasets
3. ## set cross validated KNN model object and fit it to train dataset
4. ## choose best "k" for tuned KNN model and find final RMSE

# 1.Import data

In [ ]:
import pandas as pd
import numpy as np
hitters=pd.read_csv("../input/hittlers/Hitters.csv")
df=hitters.copy()
df.columns


In [ ]:
df.head(2)

In [ ]:
df.info()

In [ ]:
df.shape



####Create X variables( from dummies ) and y variable




In [ ]:
import pandas as pd
import numpy as np
df=df.dropna()
df.shape

In [ ]:
dms=pd.get_dummies(df[['League', 'Division', 'NewLeague']])  
dms.head(2)

In [ ]:
x=df.drop(["Salary",'League', 'Division', 'NewLeague'], axis=1).astype("float64")
x.head(2)


In [ ]:
X = pd.concat([x, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X.head(2)

In [ ]:
y=df["Salary"]

 # 2.Split the data into train and test data

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test= train_test_split(X,y,test_size=0.25, random_state=42)

# 3.set cross validated KNN model object and fit it to train dataset

In [ ]:

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

knn=KNeighborsRegressor()
k_params={"n_neighbors":np.arange(1,30,1)}


knncv=GridSearchCV(knn,k_params, cv=10)
knncv.fit(X_train, y_train)


 # 4.choose best "k" for tuned KNN model and find final RMSE for test error

In [ ]:
knncv.best_params_["n_neighbors"]   #the best k for this dataset

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score 
knn_tune=KNeighborsRegressor(n_neighbors = knncv.best_params_["n_neighbors"]).fit(X_train, y_train)
np.sqrt(mean_squared_error(y_test, knn_tune.predict(X_test)))

# CROSSCHECK and exercises for different k values with not validated model

#### Let's create not validated KNN model object and fit it WİTH default n_neighbors=5  (this is our k number for now) and find RMSE




In [ ]:
# Create KNN model object with default settings (n_neighbors=5 which means k value=5)
from sklearn.neighbors import KNeighborsRegressor
knn_model=KNeighborsRegressor().fit(X_train, y_train)
knn_model


In [ ]:
knn_model.n_neighbors

In [ ]:
#Find Test Error (RMSE) within predicted y values and "test data real y values". 
#To do that, first predict y values from "test data x values". 
#In brief,from test data x values--> gain predicted y values and compare it with test data real y values
ypredict=knn_model.predict(X_test)
ypredict[0:5]

In [ ]:
#### Find Test Error (RMSE) within predicted y values and "test data y values.
from sklearn.metrics import mean_squared_error, r2_score 
np.sqrt(mean_squared_error(ypredict,y_test))

# CROSSCHECK and exercises for different k values with not validated and validated model

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
RMSE=[]     
RMSECV=[]    #rmse values with cross validation.
for k in range(10):
    k=k+1
    knn_model = KNeighborsRegressor(n_neighbors = k).fit(X_train, y_train)    #train error
    ytrainpredict=knn_model.predict(X_train)
    rmse=np.sqrt(mean_squared_error(ytrainpredict,y_train))
    rmsecv=np.sqrt(-1*cross_val_score(knn_model,X_train,y_train,cv=10, scoring="neg_mean_squared_error").mean())
    RMSE.append(rmse)
    RMSECV.append(rmsecv)
    print("for k=", k, "    RMSE value=",rmse, "     Cross Validated RMSE", rmsecv)
    #Here the result of RMSE with CV : the range of it is btw 283 and 301. It is smaller than rmse range.

### Now, we gain train error(RMSE):                   from train data
### And we gain train error(Cross Validated RMSE):    from cross validated model

## Note that,it is not important to have small RMSE which is misleading here.
## We are looking for small range. RMSE range is 179-271. CV RMSE range is 283-301.
## We always consider that test error must be calculated from validated  model.

In [ ]:
knn_tune=KNeighborsRegressor(n_neighbors = knncv.best_params_["n_neighbors"]).fit(X_train, y_train)
np.sqrt(mean_squared_error(y_test, knn_tune.predict(X_test)))

# 413 is real RMSE for test data

#### https://github.com/mvahit/DSMLBC/blob/master/README.md